In [76]:
import pandas as pd
base_file_path = "../../files"

officer_prof_df = pd.read_csv(base_file_path + "/profiles/officer_profile.csv")
uof_df = pd.read_csv(base_file_path + "/events/uof_full.csv")

def normalize_sex(sex):
  if (sex == "M" or sex == "MALE"):
    return "MALE"
  elif (sex == "F" or sex == "FEMALE"):
    return "FEMALE"
  else:
    return "NA"

def normalize_race(race):
  if race in ["WHITE", "BLACK", "HISPANIC", "ASIAN/PACIFIC ISLANDER", "NATIVE AMERICAN/ALASKAN NATIVE"]:
    return race
  elif race == "WHITE HISPANIC":
    return "HISPANIC"
  elif race in ["AMER IND/ALASKAN NATIVE", "NATIVE AMERICAN"]:
    return "NATIVE AMERICAN/ALASKAN NATIVE"
  else:
    return race

uof_df["off_sex"] = uof_df["off_sex"].transform(normalize_sex)
uof_df["off_race"] = uof_df["off_race"].transform(normalize_race)
officer_prof_df["off_sex"] = officer_prof_df["off_sex"].transform(normalize_sex)
officer_prof_df["off_race"] = officer_prof_df["off_race"].transform(normalize_race)

join_left_cols = ["off_first_name", "off_last_name", "off_birth_year", "off_sex", "off_race"]
join_right_cols = ["off_first_name", "off_last_name", "off_year_of_birth", "off_sex", "off_race"]
""" write a file containing uof events missing pieces of our join key """
uof_df[uof_df[join_left_cols].isna().any(axis=1)].to_csv(base_file_path + "/events/officer_id_merged/excluded/uof_keys_missing_filter_1.csv", index=False)
officer_prof_df[officer_prof_df[join_right_cols].isna().any(axis=1)].to_csv(base_file_path + "/events/officer_id_merged/excluded/officer_profs_keys_missing_filter_1.csv", index=False)
""" drop rows in our data sets for which some piece of the join key is null/nonexistent """
uof_df.dropna(axis="index", how="any", subset=join_left_cols, inplace=True)
officer_prof_df.dropna(axis="index", how="any", subset=join_right_cols, inplace=True)

""" merge our data """
uof_merged_df = pd.merge(uof_df, officer_prof_df, how="left", left_on=join_left_cols, right_on=join_right_cols, validate="m:m")

""" find uof incidents we've matched to more than one officer """
matches_by_uof_id = uof_merged_df.groupby("uof_id").size().reset_index(name="count")
uof_ids_multiple_matches = matches_by_uof_id[matches_by_uof_id["count"] > 1]["uof_id"].to_numpy()
row_mask = uof_merged_df.isin({'uof_id': uof_ids_multiple_matches}).any(1)
""" uof incidents matched to more than one officer """
uof_merged_df[row_mask].to_csv(base_file_path + "/events/officer_id_merged/uof_merged_double_match_filter_1.csv", index=False)
""" write a file containing uof incidents we could not match to an officer """
uof_merged_df[uof_merged_df["off_id"].isna()].to_csv(base_file_path + "/events/officer_id_merged/uof_merged_no_match_filter_1.csv", index=False)
""" write a file containing uof incidents merged with officer info """
uof_merged_df.to_csv(base_file_path + "/events/officer_id_merged/uof_merged_filter_1.csv", index=False)


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_34065/630295531.py:5: DtypeWarning: Columns (1,4,15,16,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  uof_df = pd.read_csv(base_file_path + "/events/uof_full.csv")
